# Train and Save Model

### Use transfer learning and image augmentation to train an Xception Model and save it

In [16]:
# Imports
from tensorflow import uint8, float32, cast
from tensorflow.keras.layers import Dense, Dropout, Conv2D, Dropout, GlobalAveragePooling2D, Input
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import Xception
from tensorflow.keras.applications.xception import preprocess_input
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import Model
from os import path, walk

In [2]:
# Constants
TRAIN_DIR = path.join('Fresh', 'train')
VAL_DIR = path.join('Fresh', 'val')
MODEL_DIR = path.join('model_dir', 'Xception', 'transfer-learning', 'best3')
RUNTIME_MODEL_DIR = path.join(MODEL_DIR, 'runtime')
NUM_EPOCHS = 5

In [3]:
# Get the names of the classes
class_names = []
for subdir, dirs, files in walk(TRAIN_DIR):
    try:
        class_names.append(subdir.split('\\')[2])
    except:
        pass
num_classes = len(class_names)

In [4]:
# Instantiate the base Xception model and freeze its layers
base_model = Xception(input_shape=(299, 299, 3), include_top=False)
base_model.trainable = False

In [5]:
# Append the base model with a few custom layers we will train first
model = Sequential([base_model, 
                    Conv2D(32, 3, activation='relu'),
                    Dropout(0.2),
                    GlobalAveragePooling2D(),
                    Dense(num_classes, activation='softmax', name="out")])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
xception (Model)             (None, 10, 10, 2048)      20861480  
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 8, 8, 32)          589856    
_________________________________________________________________
dropout (Dropout)            (None, 8, 8, 32)          0         
_________________________________________________________________
global_average_pooling2d (Gl (None, 32)                0         
_________________________________________________________________
out (Dense)                  (None, 34)                1122      
Total params: 21,452,458
Trainable params: 590,978
Non-trainable params: 20,861,480
_________________________________________________________________


In [6]:
# Initialize the training and validation generators
train_data_gen = ImageDataGenerator(preprocessing_function=preprocess_input,
                                    rotation_range=20,
                                    width_shift_range=0.1,
                                    height_shift_range=0.1,
                                    shear_range=10,
                                    zoom_range=0.1,
                                    horizontal_flip=True)
train_generator = train_data_gen.flow_from_directory(TRAIN_DIR,
                                                     target_size=(299,299),
                                                     color_mode='rgb',
                                                     batch_size=16,
                                                     class_mode='categorical',
                                                     shuffle=True)

val_data_gen = ImageDataGenerator(preprocessing_function=preprocess_input)
val_generator = val_data_gen.flow_from_directory(VAL_DIR,
                                                     target_size=(299,299),
                                                     color_mode='rgb',
                                                     batch_size=16,
                                                     class_mode='categorical',
                                                     shuffle=True)

Found 3262 images belonging to 34 classes.
Found 685 images belonging to 34 classes.


In [7]:
# Compile the model and the checkpoint to save only the best
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])
checkpoint = ModelCheckpoint(MODEL_DIR, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')

In [8]:
# Train the model and save only the best
model.fit(train_generator, epochs=10, validation_data=val_generator, callbacks=[checkpoint])

Epoch 1/10
  4/204 [..............................] - ETA: 1:47 - loss: 3.5578 - accuracy: 0.1562

c:\users\gregh\appdata\local\programs\python\python37\lib\site-packages\PIL\Image.py:961: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


170/204 [========================>.....] - ETA: 19s - loss: 0.9167 - accuracy: 0.7756

c:\users\gregh\appdata\local\programs\python\python37\lib\site-packages\PIL\TiffImagePlugin.py:785: UserWarning: Corrupt EXIF data.  Expecting to read 4 bytes but only got 0. 
  warnings.warn(str(msg))


204/204 [==============================] - ETA: 0s - loss: 0.8170 - accuracy: 0.7980
Epoch 00001: val_accuracy improved from -inf to 0.69489, saving model to model_dir\Xception\transfer-learning\best3
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: model_dir\Xception\transfer-learning\best3\assets
204/204 [==============================] - 190s 932ms/step - loss: 0.8170 - accuracy: 0.7980 - val_loss: 1.1954 - val_accuracy: 0.6949
Epoch 2/10
204/204 [==============================] - ETA: 0s - loss: 0.2373 - accuracy: 0.9347
Epoch 00002: val_accuracy did not improve from 0.69489
204/204 [==============================] - 153s 748ms/step - loss: 0.2373 - accuracy: 0.9347 - val_loss: 1.2694 - val_accuracy: 0.6832
Epoch 3/10
204/204 [==============================] - ETA: 0s - loss: 0.1283 - accuracy: 0.9617
Epoch 00003: val_accuracy improved from 0.69489 to 0.72409, saving model to model_dir\Xception\transfer-learning\bes

KeyboardInterrupt: 

In [9]:
# Let's take a look to see how many layers are in the base model
print("Number of layers in the base model: ", len(base_model.layers))

In [10]:
base_model.trainable = True # Unfreeze all layers

# Fine tune from this layer onwards
fine_tune_at = 100

# Freeze all the layers before the `fine_tune_at` layer
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable =  False

Number of layers in the base model:  132


In [13]:
model.compile(loss='categorical_crossentropy',
              optimizer = Adam(1e-5),
              metrics=['accuracy'])

In [14]:
# Fine tune some of the layers to squeeze out another 1% or so
model.fit(train_generator, epochs=5, validation_data=val_generator, callbacks=[checkpoint])

Epoch 1/5
204/204 [==============================] - ETA: 0s - loss: 0.2519 - accuracy: 0.9359
Epoch 00001: val_accuracy did not improve from 0.76642
204/204 [==============================] - 171s 840ms/step - loss: 0.2519 - accuracy: 0.9359 - val_loss: 1.0993 - val_accuracy: 0.7504
Epoch 2/5
204/204 [==============================] - ETA: 0s - loss: 0.0677 - accuracy: 0.9785
Epoch 00002: val_accuracy did not improve from 0.76642
204/204 [==============================] - 171s 839ms/step - loss: 0.0677 - accuracy: 0.9785 - val_loss: 0.9860 - val_accuracy: 0.7650
Epoch 3/5
204/204 [==============================] - ETA: 0s - loss: 0.0608 - accuracy: 0.9831
Epoch 00003: val_accuracy improved from 0.76642 to 0.77518, saving model to model_dir\Xception\transfer-learning\best3
INFO:tensorflow:Assets written to: model_dir\Xception\transfer-learning\best3\assets
204/204 [==============================] - 182s 893ms/step - loss: 0.0608 - accuracy: 0.9831 - val_loss: 0.9524 - val_accuracy: 0.7

KeyboardInterrupt: 

In [17]:
# Input preprocessing to create runtime model
i = Input([299, 299, 3], dtype=uint8)
x = cast(i, float32)
x = preprocess_input(x)
x = model(x)
runtime_model = Model(inputs=[i], outputs=[x])

In [18]:
# Save the Model used for inference
runtime_model.save(RUNTIME_MODEL_DIR)

INFO:tensorflow:Assets written to: model_dir\Xception\transfer-learning\best3\runtime\assets
